In [2]:
#!pip install pandas

In [14]:
import json
import pandas as pd
from collections import defaultdict
from itertools import combinations,permutations 
import numpy as np
from sklearn import cluster
import glob
import pickle
import warnings
warnings.filterwarnings("ignore")

#### Data Processing

In [15]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'split_merged_dataset/*_train.jsonl')
all_datasets
all_datasets[3].split('/')[-1][:-6].split('_')[0]

'ta'

In [16]:
languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    languages.append(lang)
print(languages)

['pa', 'gu', 'hi', 'ta', 'as', 'ml', 'te', 'kn', 'bn', 'or', 'mr', 'en']


In [17]:
ent_dict = defaultdict()
fact_dict_all = defaultdict()

languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    
    e_dict = {}
    print(lang)
    with open(dataset) as f:
        
        
        unique_facts = []

        for line in f:
    #         if i>50:
    #             break
            #print(i)
            data = json.loads(line)
            f = []
            for l in data['facts_list']:
                #print('sentence')
                f2 = []
                for fact in l:
                    f2.append(fact[0])  
                    # Generate facts 
                    unique_facts.append(fact[0])  
                f.append(f2)
                #print(f)
            e_dict[data['entity_name']] = f
            i+=1
    languages.append(lang)
    fact_dict_all[lang] = set(unique_facts)
    print(lang,len(fact_dict_all[lang]))
    ent_dict[lang] = e_dict

pa
pa 218
gu
gu 133
hi
hi 216
ta
ta 225
as
as 136
ml
ml 244
te
te 166
kn
kn 165
bn
bn 245
or
or 141
mr
mr 189
en
en 301


#### Affinity Matrix

In [505]:
#print(languages)

In [18]:
fact_dict = defaultdict()
factsetdict = defaultdict()

for lang in languages:
    print(lang)
    fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
    factsetdict[lang] = set(fact_dict_all[lang])
    
    fact_dict[lang] = fdict
    #print(fact_dict) 

pa
gu
hi
ta
as
ml
te
kn
bn
or
mr
en


In [19]:
perm_dict = defaultdict()
for i,(k,facts) in enumerate(factsetdict.items()):
    perm_dict[k] = list(permutations(facts,2))

#### PMI 

In [14]:
# data=pd.read_csv('../datasets/fact_pair_probs/en.tsv',sep='\t',header=None)
# data.columns = ['relation','scores']
# data.head(5)

In [20]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'fact_pair_probs/*.tsv')
all_datasets 

['/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ta.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/bn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/mr.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/te.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/as.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/gu.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/hi.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/en.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/pa.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/kn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/or.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ml.tsv']

In [569]:
pmi_dict = defaultdict()
for lang in languages:
    for dataset in all_datasets:
        if lang == dataset.split('/')[-1][:-6].split('_')[0]:
            pmi_list = []
            with open(dataset) as f:
                for line in f:
                    l = line.split('\t')
                    pmi_list.append(l)
    pmi_dict[lang] = pmi_list

In [523]:
#pmi_dict

#### Affinity Matrix

In [40]:
affinity_dict = defaultdict()
for lang in languages: 
    print(lang)
    a_dict = defaultdict()
    for p in perm_dict[lang]:
        #print(p)
        look = p[0] + '|' +  p[1]
        look2 = p[1] + '|' +  p[0]
        for item in pmi_dict[lang]:
            if look == item[0] or look2 == item[0]:
                # print("yes")
                #print(p)
                a_dict[p] = float(item[1].split('\n')[:-1][0])
                break
            else:
                a_dict[p] = float(0)
    affinity_dict[lang] = a_dict

te


NameError: name 'perm_dict' is not defined

#### Create affinity matrix

In [ ]:
fact_dict_

In [572]:
for lang in languages:
    print(lang)
    if lang=='en':
    s = len(fact_dict[lang]),len(fact_dict[lang])
    affinity_matrix  = np.zeros(s)
  
    for r in fact_dict[lang].keys():
        print(r)
#        print(r)
        cols = np.zeros(len(fact_dict[lang].keys()))
        for c in fact_dict[lang].keys():
            for item in affinity_dict[lang].keys():
               # print(item)
                #print(r,item[0])
                if fact_dict[lang][r] == item[0] and fact_dict[lang][c] == item[1]:
                    #print("yes")
                    affinity_matrix[r][c] = affinity_dict[lang][item]
                    affinity_matrix[c][r] = affinity_dict[lang][item]

    np.save(f'../datasets/affinity_matrices/{lang}_affinity_matrix.npy',affinity_matrix)   


pa
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
gu
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80

KeyError: 'en'

##### Spectral clustering for all entities on validation set

In [160]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/"
all_datasets = glob.glob(root  + '*_test.jsonl')
all_datasets 

['/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/te_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/gu_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/as_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/or_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ta_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/bn_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/hi_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/pa_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/en_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/mr_merged_test.jsonl',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ml_merged_test.jsonl',

In [51]:
#val_dataset = '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/' + lang + '_merged_test.jsonl

In [161]:
ent_dict = defaultdict()
#fact_dict_all = defaultdict()
cnts_dict = {}
languages = []

languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    print(lang)
    
    e_dict = {}
    
    cnts = []
    
    with open(dataset) as f:
        
        
        unique_facts = []

        for line in f:
            c = 0
            data = json.loads(line)
            f = []
            for l in data['facts_list']:
                #print('sentence')
                f2 = []
                for fact in l:
                    f2.append(fact[0])  
                    # Generate facts 
                    unique_facts.append(fact[0])  
                f.append(f2)
                #print(f)
                c+=1
            
            cnts.append(c)
    
            e_dict[data['entity_name']] = f
        
    languages.append(lang)
    #fact_dict_all[lang] = set(unique_facts)
    print(lang,len(fact_dict_all[lang]))
    ent_dict[lang] = e_dict
    cnts_dict[lang] = cnts

te
te 166
gu
gu 133
as
as 136
or
or 141
ta
ta 225
bn
bn 245
hi
hi 216
pa
pa 218
en
en 301
mr
mr 189
ml
ml 244
kn
kn 165


In [165]:
cnts_dict_ent = defaultdict()
for i,lang in enumerate(cnts_dict):
    num_sent = pd.Series(cnts_dict[lang]).sum()
    #print(lang,num_sent)
    cnts_dict_ent[lang]=num_sent
print(cnts_dict_ent)

defaultdict(None, {'te': 2459, 'gu': 706, 'as': 818, 'or': 1691, 'ta': 6548, 'bn': 13285, 'hi': 5750, 'pa': 4024, 'en': 18003, 'mr': 2506, 'ml': 6343, 'kn': 1760})


In [163]:
pd.Series(cnts_dict['te']).value_counts()

2     134
1     127
3     109
4      84
5      65
6      48
7      37
8      26
9      19
10     15
dtype: int64

##### Clustering on a dataset

In [29]:
# fact_dict = defaultdict()
# factsetdict = defaultdict()

# for lang in languages:
#     print(lang)
#     fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
#     factsetdict[lang] = set(fact_dict_all[lang])
    
#     fact_dict[lang] = fdict
#     #print(fact_dict) 

In [65]:
#fact_dict['en']

In [64]:
#fact_dict['en']

In [373]:
#fact_dict.values()

In [63]:
# #ent_dict 
# # for e,c in zip(ent_dict.keys(),cnts):
# #     print(e,c)
# root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

# entpred_dict_all = defaultdict()

# cnt =0
# nothing=0
# for lang in languages:
#     if lang == 'en':
       
#         affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
#         print("loaded affinity Matrix")

#         entpred_dict  = defaultdict()

#         for i,v in enumerate(ent_dict[lang].keys()):
#             print(lang)
#             cnt+=1
#             if cnt>10:
#                 break
#             #print(i)
#             #print(v,ent_dict[v])
#             e = ent_dict[lang][v]
#             c = cnts_dict[lang][i]
           
#             print(v,e,c)
#             #print(c)

#             flat_list = [item for sublist in e for item in sublist]
#             flat_set = set(flat_list)
#             #print(flat_set)
#             flat_ids = []
#             #print(flat_list)

#             for item in flat_set:
#                 for i,(key,value) in enumerate(fact_dict[lang].items()):
#                     if value==item:
#                         #print(value)
#                         flat_ids.append(key)
#             #print(flat_ids)
#             ent_perm_list = list(permutations(flat_ids,2))

#             ent_aff_matrix = np.zeros((len(flat_ids),len(flat_ids)))
#             #print(ent_aff_matrix.shape)


#             for p in ent_perm_list:
#                 ent_aff_matrix[flat_ids.index(p[0]),flat_ids.index(p[1])] = affinity_matrix[p[0],p[1]]

#             #print(ent_aff_matrix)


#             # Predicted clusters
#             try:
#                 #ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
#                 #print("Clusters",ent_clusters)
#                 ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
#                 print("predicted Clusters",ent_clusters)
#             except Exception:
#                 #nothing +=1
#                 #print(nothing)
#                 #print("No clusters")
#                 #print(ent_aff_matrix.shape,c)
#                 continue

                
#             # Group facts by clusters
#             d = defaultdict()
#             for i,e in zip(flat_set,ent_clusters):
#                 d[i]=e
#             print(d)

#             x = defaultdict(list)
            
#             for key, value in sorted(d.items()):
#                 x[value].append(key)

#             pred_list = list(x.values())
#             if len(pred_list) != c:
#                 print("yes")
#             print(pred_list)
        
#             entpred_dict[v] = pred_list


#         entpred_dict_all[lang] = entpred_dict

#### Label Matching on clustering

In [62]:
# root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

# cnt =0
# nothing=0
# languages = ['en']
# for lang in languages:
#     if lang == 'en':
#         print(lang)
#         cnt+=1
#         if cnt>10:
#             break
#         affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
#         print("loaded affinity Matrix")
#         print(affinity_matrix)

In [73]:
#fact_dict['en']

In [59]:
# #ent_dict 
# # for e,c in zip(ent_dict.keys(),cnts):
# #     print(e,c)
# root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

# entpred_dict_all = defaultdict()

# cnt =0
# nothing=0
# for lang in languages:
#     if lang == 'en':
#         print(lang)
#         cnt+=1
#         if cnt>10:
#             break
#         affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
#         print("loaded affinity Matrix")


#         entpred_dict  = defaultdict()

#         for i,v in enumerate(ent_dict[lang].keys()):
#             #print(i)
#             #print(v,ent_dict[v])
#             e = ent_dict[lang][v]
#             c = cnts_dict[lang][i]
           
#             print("v",v)
#             print("e",e)
#             print("c",c)
            
#             #print(flat_list)

            
#             # Get uniques facts 
            
#             flat_list = [item for sublist in e for item in sublist]
#             flat_set = set(flat_list)
#             print(flat_set)
#             flat_ids = []
            
#             for item in flat_set:
#                 for i,(key,value) in enumerate(fact_dict[lang].items()):
#                     if value==item:
#                         #print(value)
#                         flat_ids.append(key)
#             print(flat_ids)
            
            
#             # Get ids
#             for 
            
            
            
#             ent_perm_list = list(permutations(flat_ids,2))
            

#             ent_aff_matrix = np.zeros((len(flat_ids),len(flat_ids)))
#             #print(ent_aff_matrix.shape)


#             for p in ent_perm_list:
#                 ent_aff_matrix[flat_ids.index(p[0]),flat_ids.index(p[1])] = affinity_matrix[p[0],p[1]]

#             print(ent_aff_matrix)


#             # clusters
#             try:
#                 #ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
#                 #print("Clusters",ent_clusters)
#                 ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
#                 print("predicted Clusters",ent_clusters)
#             except Exception:
#                 #nothing +=1
#                 #print(nothing)
#                 #print("No clusters")
#                 #print(ent_aff_matrix.shape,c)
#                 continue

#             # Group facts by clusters
#             d = defaultdict()
#             for i,e in zip(flat_set,ent_clusters):
#                 d[i]=e
#             print(d)

#             x = defaultdict(list)
            
#             for key, value in sorted(d.items()):
#                 x[value].append(key)

#             pred_list = list(x.values())
#             if len(pred_list) != c:
#                 print("yes")
#             #print(v)

#             entpred_dict[v] = pred_list


#         entpred_dict_all[lang] = entpred_dict

#### New logic for cluster matching

In [167]:
#if fact_dict['en'].value

In [131]:
languages

['te', 'gu', 'as', 'or', 'ta', 'bn', 'hi', 'pa', 'en', 'mr', 'ml', 'kn']

In [93]:
from labelmapping import label_matching,make_cost_matrix,translate_clustering,accuracy

In [229]:
#ent_dict 
# for e,c in zip(ent_dict.keys(),cnts):
#     print(e,c)
from collections import Counter
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

entpred_dict_all = defaultdict()

cnt =0
nothing=0
macro_pr = []
macro_rec = []
macro_pr_old = []
macro_rec_old = []
facts_len = 0
weighted_acc =[]
missed = []
scores_dict = defaultdict()

for lang in languages:
    #if lang == 'te':

    scores = defaultdict()
    missed = 0

    print(lang)
    affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
    print("loaded affinity Matrix")


    entpred_dict  = defaultdict()

    for i,v in enumerate(ent_dict[lang].keys()):

        cnt+=1
#             if cnt>10:
#                 break
#        print(i)
        #print(v,ent_dict[v])
        e = ent_dict[lang][v]
        c = len(ent_dict[lang][v])

#         print("v",v)
#         print("e",e)
#         print("c",c)

        #print(flat_list)


        # Get uniques facts 

        flat_list = [item for sublist in e for item in sublist]

        flat_set = set(flat_list)
        #print(flat_set)
        flat_ids = []

        for item in flat_set:
            for i,(key,value) in enumerate(fact_dict[lang].items()):
                if value==item:
                    #print(value)
                    flat_ids.append(key)
        #print(flat_ids)


        # Get ids
        flat_list_cnts = dict(Counter(flat_list))
        #print(flat_list_cnts)

        ent = e
        for i,value in enumerate(ent):
            for j,subitem in enumerate(value):
                #print(ent[i][j])
                for lookupitem in flat_list_cnts.keys():
                    #print(lookupitem)
                    if subitem == lookupitem and flat_list_cnts[lookupitem]!=1:    
                        ent[i][j] = tuple([ent[i][j],lookupitem + '_' + str(flat_list_cnts[lookupitem])])
                        flat_list_cnts[lookupitem] -= 1
                        #print(flat_list_cnts)
                        #print(e)
                        continue
                #print(type(ent[i][j]))
                if not isinstance(ent[i][j], tuple):
                    #print(True)
                    ent[i][j] = tuple([ent[i][j],ent[i][j]])


        #print("Ent",ent)
        flat_ids = []
        sent_id=0
        act_list = []
        flag=0
        new_fact_id = 200000
        for sent in ent:
            for item in sent:
                #print("item[0]",item[0])
                for i,(key,value) in enumerate(fact_dict[lang].items()):
                    if value==item[0]:
                        flag==1
                        flat_ids.append(key)
                        act_list.append(sent_id)
                        break
#                     flat_ids.append(new_fact_id)
#                     new_fact_id +=1
#                break

            sent_id +=1

#        print("Actual Clusters",act_list)
#         print(act_list)

        ent_perm_list = list(permutations(flat_ids,2))


        ent_aff_matrix = np.zeros((len(flat_ids),len(flat_ids)))
        #print(ent_aff_matrix.shape)


        for p in ent_perm_list:
            ent_aff_matrix[flat_ids.index(p[0]),flat_ids.index(p[1])] = affinity_matrix[p[0],p[1]]

        #print(ent_aff_matrix)


        # clusters
        try:
            #ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
            #print("Clusters",ent_clusters)
            ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
            #print("predicted Clusters",ent_clusters)
        except Exception:
            #nothing +=1
            #print(nothing)
            #print("No clusters")
            #print(ent_aff_matrix.shape,c)
            continue


        # LABEL MAPPING
        if (pd.Series(act_list).nunique() == pd.Series(ent_clusters).nunique()):
            new_clusters,cm,old_acc,old_pr,old_re,new_cm,new_acc,new_pr,new_re  = label_matching(act_list,ent_clusters)
        else:
            #print("Missed entity")
            missed +=1
            print("missed")

        #print("New Clusters",new_clusters)

        macro_pr.append(new_pr)
        macro_rec.append(new_re)


        # Weighted accuracy
        #print(new_acc)
        weighted_acc.append(new_acc * len(flat_ids))
        facts_len+= len(flat_ids)

        macro_pr_old.append(old_pr)
        macro_rec_old.append(old_re)


        entpred_dict[v] = new_clusters

    scores['missed_ent'] = missed
    macro_pr_total = pd.Series(macro_pr).sum()/cnt
    macro_rec_total = pd.Series(macro_rec).sum()/cnt
    weighted_acc_total = pd.Series(weighted_acc).sum()/facts_len
    macro_pr_old_total = pd.Series(macro_pr_old).sum()/cnt
    macro_rec_old_total = pd.Series(macro_rec_old).sum()/cnt

    scores['macro_pr'] = round(macro_pr_total*100,2)
    scores['macro_rec'] = round(macro_rec_total*100,2)
    scores['weighted_acc'] = round(weighted_acc_total*100,2)
    scores['macro_pr_old'] = round(macro_pr_old_total*100,2)
    scores['macro_rec_old'] = round(macro_rec_old_total*100,2)
    print(scores)
    scores_dict[lang] = scores

te
loaded affinity Matrix
defaultdict(None, {'missed_ent': 0, 'macro_pr': 79.16, 'macro_rec': 78.4, 'weighted_acc': 64.64, 'macro_pr_old': 44.66, 'macro_rec_old': 44.55})
gu
loaded affinity Matrix
missed
missed
defaultdict(None, {'missed_ent': 2, 'macro_pr': 79.34, 'macro_rec': 78.71, 'weighted_acc': 64.81, 'macro_pr_old': 46.04, 'macro_rec_old': 45.88})
as
loaded affinity Matrix
missed
missed
defaultdict(None, {'missed_ent': 2, 'macro_pr': 79.24, 'macro_rec': 78.73, 'weighted_acc': 64.88, 'macro_pr_old': 44.76, 'macro_rec_old': 44.75})
or
loaded affinity Matrix
missed
defaultdict(None, {'missed_ent': 1, 'macro_pr': 79.16, 'macro_rec': 78.45, 'weighted_acc': 64.99, 'macro_pr_old': 42.99, 'macro_rec_old': 42.87})
ta
loaded affinity Matrix
missed
defaultdict(None, {'missed_ent': 1, 'macro_pr': 82.46, 'macro_rec': 81.94, 'weighted_acc': 67.21, 'macro_pr_old': 51.74, 'macro_rec_old': 51.66})
bn
loaded affinity Matrix
missed
defaultdict(None, {'missed_ent': 1, 'macro_pr': 78.55, 'macro_rec'

In [231]:
scores_dict

defaultdict(None,
            {'te': defaultdict(None,
                         {'missed_ent': 0,
                          'macro_pr': 79.16,
                          'macro_rec': 78.4,
                          'weighted_acc': 64.64,
                          'macro_pr_old': 44.66,
                          'macro_rec_old': 44.55}),
             'gu': defaultdict(None,
                         {'missed_ent': 2,
                          'macro_pr': 79.34,
                          'macro_rec': 78.71,
                          'weighted_acc': 64.81,
                          'macro_pr_old': 46.04,
                          'macro_rec_old': 45.88}),
             'as': defaultdict(None,
                         {'missed_ent': 2,
                          'macro_pr': 79.24,
                          'macro_rec': 78.73,
                          'weighted_acc': 64.88,
                          'macro_pr_old': 44.76,
                          'macro_rec_old': 44.75}),
             'or': d

In [232]:
cols  =['language','total_ent','total_sent_all','macro_pr','macro_rec','weighted_acc','macro_pr_old','macro_rec_old']
df = pd.DataFrame(columns=cols)

for i,lang in enumerate(scores_dict):
    #print(lang)
    df2 = [{'language': lang, 'total_ent':len(ent_dict[lang]),
            'total_sent_all':cnts_dict_ent[lang],
#           'total_sent':scores_dict[lang]['total_sent'],
#            'total_sent_cov':round(scores_dict[lang]['total_sent']/cnts_dict_ent[lang],2),
        'missed_ent':scores_dict[lang]['missed_ent'],
       'macro_pr':scores_dict[lang]['macro_pr'],
        'macro_rec':scores_dict[lang]['macro_rec'],
        'weighted_acc':scores_dict[lang]['weighted_acc']
        ,'macro_pr_old':scores_dict[lang]['macro_pr_old']
        ,'macro_rec_old':scores_dict[lang]['macro_rec_old']}]
    #print(df2)
    df2= pd.DataFrame.from_dict(df2,orient='columns')
    #print(df2)
    
    df = pd.concat([df,df2], ignore_index=True)
    #df.append(df2)
    
print(df)    

df.to_csv("../results/clustering-precisionrecall-alllang.csv",index=False)

   language total_ent total_sent_all  macro_pr  macro_rec  weighted_acc  \
0        te       664           2459     79.16      78.40         64.64   
1        gu       212            706     79.34      78.71         64.81   
2        as       189            818     79.24      78.73         64.88   
3        or       411           1691     79.16      78.45         64.99   
4        ta      2356           6548     82.46      81.94         67.21   
5        bn      3492          13285     78.55      77.60         63.10   
6        hi      2178           5750     79.54      78.76         64.15   
7        pa      1282           4024     79.19      78.49         63.95   
8        en      7554          18003     80.32      79.58         64.98   
9        mr      1270           2506     80.72      80.03         65.50   
10       ml      1966           6343     80.51      79.83         65.33   
11       kn       476           1760     80.43      79.76         65.27   

    macro_pr_old  macro_

In [179]:
# macro_pr = pd.Series(macro_pr).sum()/cnt
# macro_rec = pd.Series(macro_rec).sum()/cnt
# weighted_acc = pd.Series(weighted_acc).sum()/facts_len
# macro_pr_old = pd.Series(macro_pr_old).sum()/cnt
# macro_rec_old = pd.Series(macro_rec_old).sum()/cnt
# print("Macro Precision",(macro_pr,macro_pr_old))
# print("Macro Recall",(macro_rec,macro_rec_old))
# print("Weighted_acc",weighted_acc)

Macro Precision (0.9090909090909091, 0.37878787878787873)
Macro Recall (0.9090909090909091, 0.37878787878787873)
Weighted_acc 1.0


##### Store validation cluster predictions

In [207]:
#entpred_dict_all['te']

In [360]:
entpred_dict_all.keys()

dict_keys(['te', 'gu', 'as', 'or', 'ta', 'bn', 'hi', 'pa', 'en', 'mr', 'ml', 'kn'])

In [87]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/cluster_predictions/"
with open(root + 'cluster_predictions_all.pkl', 'wb') as f:
    pickle.dump(entpred_dict_all, f)

##### Accuracy on validation set

In [213]:
#len(entpred_dict_all['en'])

In [212]:
# scores_dict = defaultdict()
# #l = ['te']
# for i,lang in enumerate(entpred_dict_all):
#     #print(i)
#     print(lang)
    
#     total_sent = 0
#     total_right = 0 
#     total_ones = 0
#     total_gtpart = 0
#     total_gt1sent = 0
#     acc_dict = {}
#     ent_sent_correct = 0
    
#     scores = defaultdict()
    
#     for j,(k,v) in enumerate(entpred_dict_all[lang].items()):
#         #print(i)
#         #print(k)
#         act = ent_dict[lang][k]

#     #     print("Pred",k,v)
#     #     print("Act",act)

        
#         num_sent = len(v)
#         total_sent += num_sent
#         #print(num_sent)

#         pred = (v)
#         right=0;ones=0
#         gtpart_right=0

#         # only one sentence and fact
#         if num_sent==1: #and len(v[0])==1:
#             ones+=1
#             #right+=1
#             #print(pred)
            
#         else :
#             total_gt1sent +=num_sent
#             for sent in pred:
#                 for act_sent in act:
#                     if sorted(sent) == sorted(act_sent):
#                         #print(sorted(sent),sorted(act_sent))
#                         right +=1
#                         #print(right)
#                         break


#         #acc = right/num_sent
#         total_right += right


#         # Check partial or fully matched sentences 
#         d = np.zeros(len(act))


#         for sent in pred:
            
#             for i,act_sent in enumerate(act):
#                 if d[i] == 1:
#                     continue


#                 if(set(sent).issubset(set(act_sent))):
#                     d[i] = 1
#                     #print("partial match",sorted(sent),sorted(act_sent))

#                     gtpart_right +=1
#                     break

#         total_gtpart += gtpart_right                
#         #print(gtpart_right,total_gtpart) 

#         total_ones += ones

        
#         if right == num_sent:
#             ent_sent_correct +=1
            
        
#     #print(i)       
#     scores['total_sent'] = total_sent
#     scores['total_right'] = total_right
#     scores['total_ones'] = total_ones
#     scores['sent_gt1'] = total_gt1sent
#     scores['total_gtpart'] = total_gtpart
#     scores['ent_sent_correct'] = ent_sent_correct
#     scores['ent'] = j+1
    
#     print(total_sent)  
#     print(total_right)
#     print(total_ones)
#     print(total_gtpart)
    
#     scores['sub_acc']  = round(total_gtpart/total_sent,2)
#     scores['acc'] = round(total_right/total_gt1sent,2)
#     scores['ent_pc_correct'] = round(ent_sent_correct/(j+1),2)
#     scores_dict[lang] = scores

In [366]:
scores_dict['te']

defaultdict(None,
            {'total_sent': 1496,
             'total_right': 529,
             'total_ones': 140,
             'sent_gt1': 1356,
             'total_gtpart': 1169,
             'ent_sent_correct': 64,
             'ent': 536,
             'sub_acc': 0.78,
             'acc': 0.39,
             'ent_pc_correct': 0.12})

In [211]:
cols  =['language','total_ent','total_sent_all','total_sent','total_sent_cov','sent_gt1','total_right','accuracy_abs_match','acc_sub_match','ent','ent_sent_correct','ent_pc_correct']
df = pd.DataFrame(columns=cols)

for i,lang in enumerate(scores_dict):
    #print(lang)
    df2 = [{'language': lang, 'total_ent':len(ent_dict[lang]),
            'total_sent_all':cnts_dict_ent[lang],
            'total_sent':scores_dict[lang]['total_sent'],
            'total_sent_cov':round(scores_dict[lang]['total_sent']/cnts_dict_ent[lang],2),
       'sent_gt1':scores_dict[lang]['sent_gt1'],
        'total_right':scores_dict[lang]['total_right'],
        'accuracy_abs_match':scores_dict[lang]['acc']
       ,'acc_sub_match':scores_dict[lang]['sub_acc']
        ,'ent':scores_dict[lang]['ent']
        ,'ent_sent_correct':scores_dict[lang]['ent_sent_correct']
        ,'ent_pc_correct':scores_dict[lang]['ent_pc_correct']}]
    #print(df2)
    df2= pd.DataFrame.from_dict(df2,orient='columns')
    #print(df2)
    
    df = pd.concat([df,df2], ignore_index=True)
    #df.append(df2)
    
df    

KeyError: 'total_sent'

In [210]:
df 

,language,total_ent,total_sent_all,macro_pr,macro_rec,weighted_acc,macro_pr_old,macro_rec_old
0,te,664,2459,99.55,99.55,100.0,45.23,45.23
1,gu,212,706,99.20,99.20,100.0,46.35,46.35
2,as,189,818,98.78,98.78,100.0,44.58,44.58
3,or,411,1691,98.98,98.98,100.0,42.70,42.70
4,ta,2356,6548,99.56,99.56,100.0,51.59,51.59
5,bn,3492,13285,99.74,99.74,100.0,46.68,46.68
6,hi,2178,5750,99.71,99.71,100.0,49.70,49.70
7,pa,1282,4024,99.65,99.65,100.0,49.66,49.66
8,en,7554,18003,99.75,99.75,100.0,55.28,55.28
9,mr,1270,2506,99.76,99.76,100.0,56.43,56.43


In [372]:
df.to_csv("../results/clustering-scores-alllang-occ.csv",index=False)